In [9]:
import requests
import pandas as pd
from google.cloud import bigquery

# Overpass Query
query = """
[out:json][timeout:25];
(
  way(around:2000,52.521909,13.413265)
  ["highway"~"path|track|pedestrian"];
);
foreach(
  way._(if:length() > 300);
  out body;
  >;
  out skel qt;
);
"""

# Fetch Data
url = "https://overpass-api.de/api/interpreter"
response = requests.get(url, data={"data": query})
data = response.json().get("elements", [])

# Separate Nodes and Ways
ways = []
nodes = {}

for element in data:
    if element["type"] == "way":
        ways.append({
            "way_id": element["id"],
            "nodes": element["nodes"],  # List of node IDs
            "highway": element["tags"].get("highway"),
            "surface": element["tags"].get("surface"),
            "bicycle": element["tags"].get("bicycle"),
            "area": element["tags"].get("area"),
            "lit": element["tags"].get("lit"),
            "smoothness": element["tags"].get("smoothness"),
            "name": element["tags"].get("name"),
            "wikidata": element["tags"].get("wikidata"),
            "wikimedia_commons": element["tags"].get("wikimedia_commons"),
            "wikipedia": element["tags"].get("wikipedia"),
            "lit": element["tags"].get("lit")
        })
    elif element["type"] == "node":
        nodes[element["id"]] = {
            "node_id": element["id"],
            "lat": element["lat"],
            "lon": element["lon"]
        }

# Convert to DataFrames
ways_df = pd.DataFrame(ways)
nodes_df = pd.DataFrame(nodes.values())

In [8]:
from collections import Counter

# Collect tag keys and count occurrences
tag_counter = Counter()

for element in data:
    if "tags" in element:
        tag_counter.update(element["tags"].keys())

# Output the tag keys and their counts
for tag_key, count in tag_counter.items():
    print(f"{tag_key}: {count}")

area: 8
bicycle: 4
highway: 11
lit: 10
smoothness: 9
surface: 11
motor_vehicle: 1
motorcar: 1
name: 6
oneway: 1
postal_code: 2
width: 1
man_made: 1
name:etymology:wikidata: 2
place: 1
wikidata: 4
wikimedia_commons: 3
wikipedia: 3
description: 1
old_name: 1


,node_id,lat,lon
0,540939123,52.520726,13.390285
1,540939126,52.520319,13.390355
2,540939771,52.520409,13.391767
3,540940252,52.520711,13.390043
4,1588122182,52.520367,13.391111
...,...,...,...
356,11394748904,52.519005,13.408882
357,1368438971,52.529777,13.425606
358,1368438973,52.529863,13.425728
359,1368438981,52.532037,13.428189


In [10]:

# Check for a successful response
if response.status_code == 200:
    result = response.json()
    # Print the formatted JSON response
    print("Extract complete")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Save the JSON to a file
with open("output.json", "w") as f:
    json.dump(result, f, indent=2)


Extract complete


NameError: name 'json' is not defined